In [ ]:
# Evaluation

In [ ]:
import pandas as pd

# MMD vs. MMD_Feature

In [ ]:
# Create a new DataFrame as a copy of the original
df_mmd_mmd2 = pd.read_csv('../evaluation/results.csv')

# Relabel the rules column in the new DataFrame
df_mmd_mmd2.loc[(df_mmd_mmd2['learner'] == 3) & (df_mmd_mmd2['rules'] == 'induction') & (df_mmd_mmd2['coverage'] == 0.6), 'rules'] = 'mmd2'
df_mmd_mmd2.loc[(df_mmd_mmd2['learner'] == 0) & (df_mmd_mmd2['rules'] == 'induction') & (df_mmd_mmd2['coverage'] == 0.6), 'rules'] = 'mmd'

# Filter rows where rules is either 'induction' or 'mmd2'
df_mmd_mmd2 = df_mmd_mmd2[df_mmd_mmd2['rules'].isin(['mmd', 'mmd2'])]

# df_mmd_mmd2.reset_index()
# Display the new DataFrame
# df_mmd_mmd2[df_mmd_mmd2['rules'].isin(['mmd', "mmd2"])]
# df_mmd_mmd2.head()

In [ ]:
# Calculate F-score
df_mmd_mmd2['fscore'] = 2 * (df_mmd_mmd2['precision_with'] * df_mmd_mmd2['recall_with']) / (df_mmd_mmd2['precision_with'] + df_mmd_mmd2['recall_with'])
# df_mmd_mmd2.head()

In [ ]:
df_mmd_mmd2

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

# Create the box plot
plt.figure(figsize=(10, 4), dpi=500)
plt.rcParams['text.usetex'] = True

# Define the colors
palette = {'mmd2': 'darkorange', 'mmd': 'lightsteelblue'}

sns.boxplot(x='blackbox', y='fscore', hue='rules', data=df_mmd_mmd2, palette=palette)
# plt.title('F-score Distribution by Subject for MMD and Our Approach')
plt.grid(axis="y", linestyle='--',)
plt.grid(axis="x", linestyle='--',which='both')
plt.xlabel(r'$Subjects$')
plt.ylabel(r'$F-Score$')
plt.yticks(np.arange(0, 0.81, 0.2))

plt.xticks(rotation=45)
first_patch = mpatches.Patch(ec="black" ,facecolor='lightsteelblue', label='Decision Tree')
second_patch = mpatches.Patch(ec="black" ,facecolor='darkorange',label=r'$MMD_{Feature}$')

plt.legend(title=r'\textbf{Approach}', labels=[r'$MMD$', r'$MMD_{Feature}$'], handles=[first_patch,second_patch])
# plt.legend(handles=[first_patch, second_patch])
plt.tight_layout()
plt.savefig('./RQ1-fscore.pdf')

### Statistical Analysis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

# Get unique subjects
subjects = df_mmd_mmd2['blackbox'].unique()

# Initialize results list
results = []

# Perform Mann-Whitney U test for each subject
for subject in subjects:
    mmd = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd') & (df_mmd_mmd2['blackbox'] == subject)]['fscore']
    mmd2 = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd2') & (df_mmd_mmd2['blackbox'] == subject)]['fscore']
    
    if len(mmd) > 1 and len(mmd2) > 1:  # Ensure there are enough samples
        # u_stat, p_value = mannwhitneyu(mmd2, mmd, alternative='greater')
        u_stat, p_value = mannwhitneyu(mmd2, mmd, alternative='two-sided')
        result = {
            'subject': subject,
            'u_statistic': u_stat,
            'p_value': p_value,
            'significant': p_value < 0.05
        }
        results.append(result)

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display results
# import ace_tools as tools; tools.display_dataframe_to_user(name="Mann-Whitney U Test Results by Subject", dataframe=results_df)

print(results_df)

In [ ]:
# New

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

# Get unique subjects
subjects = df_mmd_mmd2['blackbox'].unique()

# Initialize results list
results = []

for subject in subjects:
    mmd = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd') & (df_mmd_mmd2['blackbox'] == subject)]['time_with']
    mmd2 = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd2') & (df_mmd_mmd2['blackbox'] == subject)]['time_with']
    mmd_precision = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd') & (df_mmd_mmd2['blackbox'] == subject)]['specificity_with']
    mmd2_precision = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd2') & (df_mmd_mmd2['blackbox'] == subject)]['specificity_with']
    mmd_recall = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd') & (df_mmd_mmd2['blackbox'] == subject)]['recall_with']
    mmd2_recall = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd2') & (df_mmd_mmd2['blackbox'] == subject)]['recall_with']
    
    if len(mmd) > 1 and len(mmd2) > 1:  # Ensure there are enough samples
        u_stat, p_value = mannwhitneyu(mmd2, mmd, alternative='less')
        
        mmd_min = mmd.min()
        mmd_max = mmd.max()
        mmd_median = mmd.median()
        
        mmd2_min = mmd2.min()
        mmd2_max = mmd2.max()
        mmd2_median = mmd2.median()
        
        improvement_factor = round((mmd_median / mmd2_median), 1) if mmd2_median != 0 else float('inf')
        
        mmd_precision_median = mmd_precision.median()
        mmd2_precision_median = mmd2_precision.median()
        mmd_recall_median = mmd_recall.median()
        mmd2_recall_median = mmd2_recall.median()
        
        result = {
            'subject': subject,
            'u_statistic': u_stat,
            'p_value': p_value,
            'significant': p_value < 0.05,
            'mmd_min': mmd_min,
            'mmd_median': mmd_median,
            'mmd_max': mmd_max,
            'mmd2_min': mmd2_min,
            'mmd2_median': mmd2_median,
            'mmd2_max': mmd2_max,
            'improvement_factor': improvement_factor,
            'mmd_specificity_median': mmd_precision_median,
            'mmd2_specificity_median': mmd2_precision_median,
            'mmd_recall_median': mmd_recall_median,
            'mmd2_recall_median': mmd2_recall_median
        }
        results.append(result)

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)
results_df

### Time

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

# Create the box plot
plt.figure(figsize=(10, 4), dpi=500)
plt.rcParams['text.usetex'] = True

# Define the colors
palette = {'mmd2': 'darkorange', 'mmd': 'lightsteelblue'}

sns.boxplot(x='blackbox', y='time_with', hue='rules', data=df_mmd_mmd2, palette=palette)
# plt.title('F-score Distribution by Subject for MMD and Our Approach')
plt.grid(axis="y", linestyle='--',)
plt.grid(axis="x", linestyle='--',which='both')
plt.xlabel(r'$Subjects$')
plt.ylabel(r'$Time$ $in$ $Seconds$ $(log)$')
plt.yscale('log')

#plt.xticks(rotation=45)
first_patch = mpatches.Patch(ec="black" ,facecolor='lightsteelblue', label='Decision Tree')
second_patch = mpatches.Patch(ec="black" ,facecolor='darkorange',label=r'$MMD_{feature}$')

plt.legend(title=r'\textbf{Approach}', labels=[r'$MMD$', r'$MMD_{feature}$'], handles=[first_patch,second_patch])
# plt.legend(handles=[first_patch, second_patch])
plt.tight_layout()
plt.savefig('./RQ1-time.pdf')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

# Get unique subjects
subjects = df_mmd_mmd2['blackbox'].unique()

# Initialize results list
results = []

for subject in subjects:
    mmd = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd') & (df_mmd_mmd2['blackbox'] == subject)]['time_with']
    mmd2 = df_mmd_mmd2[(df_mmd_mmd2['rules'] == 'mmd2') & (df_mmd_mmd2['blackbox'] == subject)]['time_with']
    
    if len(mmd) > 1 and len(mmd2) > 1:  # Ensure there are enough samples
        u_stat, p_value = mannwhitneyu(mmd2, mmd, alternative='less')
        
        mmd_min = mmd.min()
        mmd_max = mmd.max()
        mmd_median = mmd.median()
        
        mmd2_min = mmd2.min()
        mmd2_max = mmd2.max()
        mmd2_median = mmd2.median()
        
        improvement_factor = round((mmd_median / mmd2_median), 1) if mmd2_median != 0 else float('inf')
        
        result = {
            'subject': subject,
            'u_statistic': u_stat,
            'p_value': p_value,
            'significant': p_value < 0.05,
            'mmd_min': mmd_min,
            'mmd_median': mmd_median,
            'mmd_max': mmd_max,
            'mmd2_min': mmd2_min,
            'mmd2_median': mmd2_median,
            'mmd2_max': mmd2_max,
            'improvement_factor': improvement_factor
        }
        results.append(result)

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)
results_df
# Display results
#latex = results_df.drop(["u_statistic", "significant"], axis=1)
#print(latex.to_latex(index=False))

# MMD vs. MMD Feature vs. Tree

In [ ]:
# Create a new DataFrame as a copy of the original
df_tree = pd.read_csv('../evaluation/results.csv')

# Relabel the rules column in the new DataFrame
df_tree.loc[(df_tree['learner'] == 3) & (df_tree['rules'] == 'induction') & (df_tree['coverage'] == 0.6), 'rules'] = 'mmd2'
df_tree.loc[(df_tree['learner'] == 0) & (df_tree['rules'] == 'induction') & (df_tree['coverage'] == 0.6), 'rules'] = 'mmd'

df_tree.loc[(df_tree['learner'] == 3) & (df_tree['rules'] == 'tree') & (df_tree['coverage'] == 0.6), 'rules'] = 'tree_comp'
#df_tree.loc[(df_tree['learner'] == 0) & (df_tree['rules'] == 'tree') & (df_tree['coverage'] == 0.6), 'rules'] = 'tree_comp_0'
#df_tree.loc[(df_tree['learner'] == 6) & (df_tree['rules'] == 'tree') & (df_tree['coverage'] == 0.6), 'rules'] = 'tree_comp_4'


# Filter rows where rules is either 'induction' or 'mmd2'
#df_tree = df_tree[df_tree['rules'].isin(['mmd', 'mmd2', 'tree_comp', "tree_comp_0", "tree_comp_4"])]
df_tree = df_tree[df_tree['rules'].isin(['mmd', 'mmd2', 'tree_comp'])]

# df_tree.reset_index()
# df_tree.head(20)

In [ ]:
# Calculate F-score
df_tree['fscore'] = 2 * (df_tree['precision_with'] * df_tree['recall_with']) / (df_tree['precision_with'] + df_tree['recall_with'])
# df_tree.head()

In [ ]:
df_tree = df_tree[df_tree['rules'].isin(['mmd', 'tree_comp'])]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

# Create the box plot
plt.figure(figsize=(10, 4), dpi=500)
plt.rcParams['text.usetex'] = True

# Define the colors
palette = {'tree_comp': 'darkorange', 'mmd': 'lightsteelblue'}

sns.boxplot(x='blackbox', y='fscore', hue='rules', data=df_tree, palette=palette)
# plt.title('F-score Distribution by Subject for MMD and Our Approach')
plt.grid(axis="y", linestyle='--',)
plt.grid(axis="x", linestyle='--',which='both')
plt.xlabel(r'$Subjects$')
plt.ylabel(r'$F-Score$')
#plt.xticks(rotation=45)
plt.yticks(np.arange(0, 0.81, 0.2))

first_patch = mpatches.Patch(ec="black" ,facecolor='lightsteelblue')
second_patch = mpatches.Patch(ec="black" ,facecolor='darkorange')

plt.legend(title=r'\textbf{Approach}', labels=[r'$MMD$', r'$Tree_{feature}$'], handles=[first_patch,second_patch])
# plt.legend(handles=[first_patch, second_patch])
plt.tight_layout()
plt.savefig('./RQ2-fscore.pdf')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

# Get unique subjects
subjects = df_tree['blackbox'].unique()

# Initialize results list
results = []

# Perform Mann-Whitney U test for each subject
for subject in subjects:
    mmd = df_tree[(df_tree['rules'] == 'mmd') & (df_tree['blackbox'] == subject)]['fscore']
    tree = df_tree[(df_tree['rules'] == 'tree_comp') & (df_tree['blackbox'] == subject)]['fscore']
    
    if len(mmd) > 1 and len(tree) > 1:  # Ensure there are enough samples
        u_stat, p_value = mannwhitneyu(tree, mmd, alternative='greater')
        result = {
            'subject': subject,
            'u_statistic': u_stat,
            'p_value': p_value,
            'significant': p_value < 0.05
        }
        results.append(result)

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display results
# import ace_tools as tools; tools.display_dataframe_to_user(name="Mann-Whitney U Test Results by Subject", dataframe=results_df)

print(results_df)

## MMD vs. Tree Time

In [ ]:
# Filter the DataFrame to include only 'tree' and 'induction' approaches
df_tree = df_tree[df_tree['rules'].isin(['mmd', 'tree_comp'])]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

# Create the box plot
plt.figure(figsize=(10, 4), dpi=500)
plt.rcParams['text.usetex'] = True

# Define the colors
palette = {'tree_comp': 'darkorange', 'mmd': 'lightsteelblue'}

sns.boxplot(x='blackbox', y='time_with', hue='rules', data=df_tree, palette=palette)
# plt.title('F-score Distribution by Subject for MMD and Our Approach')
plt.grid(axis="y", linestyle='--',)
plt.grid(axis="x", linestyle='--',which='both')
plt.xlabel(r'$Subjects$')
plt.ylabel(r'$Time$ $in$ $Seconds$ $(log)$')
plt.yscale('log')
#plt.xticks(rotation=45)

first_patch = mpatches.Patch(ec="black" ,facecolor='lightsteelblue')
second_patch = mpatches.Patch(ec="black" ,facecolor='darkorange')

plt.legend(title=r'\textbf{Approach}', labels=[r'$MMD$', r'$Tree_{feature}$'], handles=[first_patch,second_patch])
# plt.legend(handles=[first_patch, second_patch])
plt.tight_layout()
plt.savefig('./RQ2-time.pdf')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

# Get unique subjects
subjects = df_tree['blackbox'].unique()

# Initialize results list
results = []

# Perform Mann-Whitney U test for each subject
for subject in subjects:
    mmd = df_tree[(df_tree['rules'] == 'mmd') & (df_tree['blackbox'] == subject)]['time_with']
    tree = df_tree[(df_tree['rules'] == 'tree_comp') & (df_tree['blackbox'] == subject)]['time_with']
    
    if len(mmd) > 1 and len(tree) > 1:  # Ensure there are enough samples
        u_stat, p_value = mannwhitneyu(tree, mmd, alternative='less')
        
        mmd_min = mmd.min()
        mmd_max = mmd.max()
        mmd_median = mmd.median()
        
        tree_min = tree.min()
        tree_max = tree.max()
        tree_median = tree.median()
        
        improvement_factor = round((mmd_median / tree_median) * 100, 1) if mmd_median != 0 else float('inf')
        
        result = {
            'subject': subject,
            'u_statistic': u_stat,
            'p_value': p_value,
            'significant': p_value < 0.05,
            'mmd_min': mmd_min,
            'mmd_max': mmd_max,
            'mmd_median': mmd_median,
            'tree_min': tree_min,
            'tree_max': tree_max,
            'tree_median': tree_median,
            'improvement_factor': improvement_factor
        }
        results.append(result)

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)

results_df

In [ ]:
# Display results
latex = results_df.drop(["u_statistic", "significant"], axis=1)
print(latex.to_latex(index=False))